In [ ]:
import pandas as pd
import numpy as np
import random
import requests     # library to handle requests

from geopy.geocoders import Nominatim  # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image
from IPython.core.display import HTML

# transforming JSON file into pandas dataframe
from pandas.io.json import json_normalize

# to visualize the results
import folium


In [ ]:
CLIENT_ID = 'client-id' # foursquare-ID 
CLIENT_SECRET = 'client-secret' # forsquare-secret
VERSION = '20180604'
LIMIT = 30

In [ ]:
address = '102 North End Ave, New York, NY'

# address = 'Manek Chowk, Ahmedabad, Ahmadabad City Taluka, Ahmedabad District, Gujarat, 380001, India'
geolocator = Nominatim(user_agent='foursquare_agent')
location = geolocator.geocode(address)
print(location)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

# 1. Search for a specific venue category


In [ ]:
search_query = 'Italian'
radius = 1000

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION,search_query, radius, LIMIT)
url

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of json to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head(5)

In [ ]:
print(dataframe['name'])

In [ ]:
# keep only columns that include venue name and anything associated with location

filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]
dataframe_filtered['categories'][0]

In [ ]:
#  function to extracts the category of the venue

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    



In [ ]:
# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

In [ ]:
# clean column names by keeping only last term
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

In [ ]:
dataframe_filtered.head()

### visualisation of our data

In [ ]:
# generate map
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# Add a red circle to our current position
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Our Location',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.4
).add_to(venues_map)

# add all Italian food restaurants as blue markers
for lat, long, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, long],
        radius=5,
        color='blue',
        popup=label,
        fill=True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

venues_map


# 2. Explore a Given Venue

In [ ]:
# We want to explore Harry's Italian Pizza bar
venue_id = '4a4560a7f964a5201aa81fe3'  # id for Harry's Italian Pizza Bar
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

In [ ]:
result = requests.get(url).json()
print(result['response']['venue'].keys())
print(result['response']['venue'])

In [ ]:
# to get Ratings of the venue
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')
    

In [ ]:
# to check the Ratings of another venue
venue_id = '4f3232e219836c91c7bfde94'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

In [ ]:
# since that venue has no ratings, we check third venue
venue_id = '528bf16711d2b7722da6b51c' #Id for pisillo Italian Palini
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

In [ ]:
# since above restaurant has higher ratings let's explore it
result['response']['venue']['tips']['count'] #Number of tips

In [ ]:
limit = 96 #Limit for number of tips
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

result = requests.get(url).json()
result

In [ ]:
tips = result['response']['tips']['items']
tips
tip = result['response']['tips']['items'][0]
tip
tip.keys()

In [ ]:
# to Display all tips
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)
tips_df
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]
tips_filtered

# 3.Search for a Foursquare User

In [ ]:
user_id = '484542633'

url = 'https://api.foursquare.com/v2/users/{}?client_id={}&client_secret={}&v={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()


In [ ]:
user_data = result['response']['user']
user_data.keys()

In [ ]:
print('First Name: ' + user_data['firstName'])
print('Last Name: ' + user_data['lastName'])
print('Home City: ' + user_data['homeCity'])

In [ ]:
user_data['tips']

In [ ]:
url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(user_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

result = requests.get(url).json()
result
tips = result['response']['tips']['items']
tips

In [ ]:
# convert result into pandas dataframe
pd.set_option('display.max_colwidth', -1)

tips_df = json_normalize(tips)

tips_df.head()
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id']
filtered_tips_df = tips_df.loc[:, filtered_columns]
filtered_tips_df.sort_values(by='agreeCount', ascending=False).head()

In [ ]:
# To get the venue for the tip with the highest value of agree count

tip_id = '5ab42b987dc9e17930e5ff5b'

url = 'http://api.foursquare.com/v2/tips/{}?client_id={}&client_secret={}&v={}'.format(tip_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
# print(result['response']['tip'].keys())
# print(result['response']['tip']['venue'])
print(result['response']['tip']['venue']['name'])
print(result['response']['tip']['venue']['location'])


In [ ]:
# print(user_data.keys())
print(user_data['friends'])
print(user_data['friends']['groups'][0]['items'])

In [ ]:
# to Retrive the Profile Image of a user

# user_data
# user_data['photo']['suffix'][1:]
photo_url = user_data['photo']['prefix'] + user_data['photo']['suffix'][1:]
Image(url=photo_url)

# 4. Explore a location

In [ ]:
latitude = 40.715337
longitude = -74.008848

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()

In [ ]:
print('These are {} around Ecco restaurant'.format(len(results['response']['groups'][0]['items'])))
# results

In [ ]:
items = results['response']['groups'][0]['items']
items[0]

In [ ]:
# Convert items to dataframe

dataframe = json_normalize(items)
dataframe.head()

filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] +['venue_id']
filtered_dataframe = dataframe.loc[:, filtered_columns]
filtered_dataframe

In [ ]:
filtered_dataframe['venue.categories'] = filtered_dataframe.apply(get_category_type, axis=1)
filtered_dataframe.columns = [col.split('.')[-1] for col in filtered_dataframe.columns]

In [ ]:
filtered_dataframe.columns

In [ ]:
venues_map = folium.Map([latitude, longitude], zoom_start=17)

# Red Circle for our current location
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Current Location',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity='1'
).add_to(venues_map)

# all venues as blue circles
for lat, long, label in zip(filtered_dataframe.lat, filtered_dataframe.lng, filtered_dataframe.categories):
    folium.CircleMarker(
        [lat, long],
        radius=10,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity='0.6'
    ).add_to(venues_map)
venues_map

In [ ]:
url = 'https://api.foursquare.com/v2/venues/trending?client_id={}&client_secret={}&ll={},{}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION)

results = requests.get(url).json()
results

In [ ]:
if len(results['response']['venues']) == 0:
    print("No trending venues available at the moment")
else:
    trending_venues = results['response']['venues']
    trending_venues_df = json_normalize(trending_venues)

    columns_filtered = ['name', 'categories'] + ['location.distance', 'location.city', 'location.postalCode', 'location.state', 'location.country', 'location.lat', 'location.lng']
    trending_venues_df = trending_venues_df.loc[:, columns_filtered]

    trending_venues_df['categories'] = trending_venues_df.apply(get_category_type, axis=1)

In [ ]:
if len(results['response']['venues']) == 0:
    venues_map = 'Cannot generate visual as no trending venues are available at the moment!'

else:
    venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

    folium.features.CircleMarker(
        [latitude, longitude],
        radius=10,
        popup='Ecco',
        fill=True,
        color='red',
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

    for lat, lng, label in zip(trending_venues_df['location.lat'], trending_venues_df['location.lng'], trending_venues_df['name']):
        folium.features.CircleMarker(
            [lat, lng],
            radius=5,
            poup=label,
            fill=True,
            color='blue',
            fill_color='blue',
            fill_opacity=0.6
        ).add_to(venues_map)
        
venues_map